In [6]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import importlib
import numpy as np
from scipy.linalg import eigh

%matplotlib ipympl
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
# np.set_printoptions(linewidth=200, formatter={'float': '{:.3e}'.format})
np.set_printoptions(linewidth=200, formatter={'float': '{:.6f}'.format})

In [7]:
# Fundamental constants
h    = 6.62607015e-34
GHz  = 1e9
fF = 1e-15
nH = 1e-9
e0   = 1.602176634e-19
Φ_0 = h/(2*e0)

In [8]:
# Circuit parameters
Csh = 15
C   = 15
Cg  = 10
CJ  = 3
Lq  = 25
Lr  = 10
Δ   = 0.1
EJ  = 10.0
l = Lq*(Lq+4*Lr) - 4*Δ**2

In [9]:
Cf = C/2 + Csh + CJ
Lf = l/(Lq+4*Lr)
C_f = Cf * 1e-15
L_f = Lf * 1e-9
Z_f = np.sqrt(L_f/C_f)

L_r = l/Lq * 1e-9
C_r = C/2 * 1e-15
Z_r = np.sqrt(L_r/C_r)

In [10]:
nmax_r = 10
nmax_f = 10
Cc=1

In [11]:
C_R = C/2
C_C = Cc
C_F = C/2 + Csh + CJ
C_mat = np.array([[C_R + C_C/2, 0, -C_C / 2,        0],
                  [0, C_F + C_C/2,       0 , -C_C / 2],
                  [-C_C / 2,        0, C_R + C_C/2, 0],
                  [       0, -C_C / 2, 0, C_F + C_C/2]])

C_inv = np.linalg.inv(C_mat)
fF = 1e-15

resonator = sq_ext.KIT_resonator(C_R_eff=C_inv[0,0]**-1, Lq = Lq, Lr = Lr, Δ = Δ, trunc_res =nmax_r)
fluxonium = sq_ext.KIT_fluxonium(C_F_eff=C_inv[1,1]**-1, Lq = Lq, Lr = Lr, Δ = Δ, trunc_flux=nmax_f)

In [12]:
fluxonium.diag(2)
resonator.diag(2);

In [51]:
# Parameters
ω_f = fluxonium.efreqs[1]-fluxonium.efreqs[0]  # Fluxonium frequency
ω_r = resonator.efreqs[1]-resonator.efreqs[0]  # Resonator frequency

Φ_f = fluxonium.flux_op(0)[0,1]
Φ_r = resonator.flux_op(0)[0,1]
g_Φ = 2 * Δ / (l * nH) * Φ_f * Φ_r / 2 / np.pi / GHz

# Pauli matrices for the fluxonium
sigma_x = np.array([[0, 1], [1, 0]])
sigma_y = np.array([[0,-1], [1, 0]]) * 1j
sigma_z = np.array([[1, 0], [0,-1]])

sigma_p = np.array([[0, 1], [0, 0]])
sigma_m = np.array([[0, 0], [1, 0]])

# Resonator operators
def create(n):
    return np.diag(np.sqrt(np.arange(1, n)), -1)

def annihilate(n):
    return np.diag(np.sqrt(np.arange(1, n)), 1)

# Number of states for the resonator (0, 1, 2)
n = 3

# Creation and annihilation operators for the resonator
a = create(n)
a_dagger = annihilate(n)

# Identity matrices
I_resonator = np.eye(n)
I_fluxonium = np.eye(2)

# Isolated Hamiltonians
H_fluxonium = ω_f / 2 * sigma_z
H_resonator = ω_r * np.dot(a_dagger, a)

# Construct the system Hamiltonian with tensor products
H_system = np.kron(H_resonator, I_fluxonium) + np.kron(I_resonator, H_fluxonium)
H_interaction = g_Φ * np.kron(a + a_dagger, sigma_x)

# Circuit Hamiltonian
H_circuit = H_system + H_interaction

I_circuit = np.eye(len(H_circuit))
a_sys = np.kron(a, I_fluxonium)
a_dagger_sys = np.kron(a_dagger, I_fluxonium)
sigma_y_sys = np.kron(I_resonator, sigma_y)

# Coupling Hamiltonian
Q_f = fluxonium.charge_op(0)[0,1]
Q_r = resonator.charge_op(0)[0,1]
C_ff = C_inv[0,2]**-1
C_rr = C_inv[1,3]**-1
H_coupling = Q_r**2/(C_rr*fF) * np.kron(a_dagger_sys-a_sys, a_dagger_sys-a_sys) + Q_f**2/(C_ff*fF) * np.kron(sigma_y_sys, sigma_y_sys)
H_coupling /= 2 * np.pi * GHz

H_total = np.kron(H_circuit,I_circuit) + np.kron(I_circuit, H_circuit) + H_coupling

In [52]:
# Find the eigenvalues (energy levels)
energies = eigh(H_total, eigvals_only=True)

energies-energies[0]

array([0.000000, 1.375827, 2.648588, 4.024415, 6.993147, 6.993228, 8.368973, 8.369053, 9.641722, 9.641805, 11.017548, 11.017628, 13.945178, 13.986366, 14.027627, 15.320986, 15.362188, 15.403464,
       16.593729, 16.634929, 16.676208, 17.969565, 18.010751, 18.052015, 20.959214, 21.000414, 22.334999, 22.376267, 23.607732, 23.649001, 24.983586, 25.024786, 27.973160, 29.348977, 30.621703,
       31.997520])

In [5]:
# Find the eigenvalues (energy levels)
energies = eigh(H_total, eigvals_only=True)

energies-energies[0]

NameError: name 'eigh' is not defined

In [47]:
qubit.diag(8)
qubit.efreqs-qubit.efreqs[0]

array([0.000000, 1.945361, 6.161082, 7.208451, 9.153874, 10.792941, 13.369603, 14.416902])